In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os
import sys
import pysam
import isabl_cli as ii

sys.path.append("../python_scripts")
from utils import *

BASEDIR = "/data1/shahs3/users/sunge/cnv_simulator"
DATADIR = f"{BASEDIR}/data"

os.environ["ISABL_API_URL"] = "https://isabl.shahlab.mskcc.org/api/v1/"

## Normal cell filtering and grouping

1. Filter out normal cells from SPECTRUM dataset and assign groups each with 700 ~ 1000 cells each. <br>

    **Format of cell IDs**

    SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM-130057A-R13-C13
    [patient id]_[sample id]-[library_id]-[row #]-[col #]
    * patient_id = SPECTRUM-OV-002
    * sample_id = S1_INFRACOLIC_OMENTUM
    * library_id = 130057A
    * row # = R13
    * col # = C13

    **Filter normal cells by:**
    * `is_normal == True` and `is_s_phase == False` 

    **Important columns:**
    * cell_id
    * sample_id
    * library_id
    * patient_id
    * aliquot_id
    * condition
    * estimated_library_size
    * total_mapped_reads
    * total_reads
    * is_s_phase
    * is_wgd
    * is_aberrant_normal_cell
    * is_normal

2. Save csv with following info in `data/all_normal_samples.csv`: <br>
    * aliquot_id (different from sample ID, one sample can have multiple aliquots)
    * bam_path
    * sample_id (name used on bam file)
    * total_cell_count
    * normal_cell_count
    * sample_group

3. For each sample, save a separate text file in `data/normal_cell_barcodes` containing barcodes of all normal cells in sample





In [3]:
# Read in table with all cell annotations
filtered_cell_table = pd.read_csv(f"{DATADIR}/filtered_cell_table.csv.gz")
print(list(filtered_cell_table.columns))

# Select columns of interest
columns_to_keep = [
    "cell_id", "sample_id", "library_id", "patient_id", "aliquot_id", "condition",
    "estimated_library_size", "total_mapped_reads", "total_reads",
    "coverage_depth", "coverage_breadth", "expected", "aligned",
    "is_s_phase", "is_normal", "is_wgd"
    ]

filtered_cell_subtable = filtered_cell_table[columns_to_keep]
print(filtered_cell_subtable.shape)
filtered_cell_subtable.head()

['cell_id', 'multiplier', 'MSRSI_non_integerness', 'MBRSI_dispersion_non_integerness', 'MBRSM_dispersion', 'autocorrelation_hmmcopy', 'cv_hmmcopy', 'empty_bins_hmmcopy', 'mad_hmmcopy', 'mean_hmmcopy_reads_per_bin', 'median_hmmcopy_reads_per_bin', 'std_hmmcopy_reads_per_bin', 'total_mapped_reads_hmmcopy', 'total_halfiness', 'scaled_halfiness', 'mean_state_mads', 'mean_state_vars', 'mad_neutral_state', 'breakpoints', 'mean_copy', 'state_mode', 'log_likelihood', 'true_multiplier', 'paired_mapped_reads', 'sample_id', 'paired_duplicate_reads', 'estimated_library_size', 'pick_met', 'expected', 'index_i5', 'condition', 'library_id', 'is_control', 'total_mapped_reads', 'total_reads', 'standard_deviation_insert_size', 'unpaired_mapped_reads', 'fastqscreen_salmon', 'unpaired_duplicate_reads', 'fastqscreen_human', 'percent_duplicate_reads', 'row', 'fastqscreen_nohit', 'is_contaminated', 'total_properly_paired', 'overlap_with_all_filters', 'coverage_breadth', 'fastqscreen_human_multihit', 'index_i

,cell_id,sample_id,library_id,patient_id,aliquot_id,condition,estimated_library_size,total_mapped_reads,total_reads,coverage_depth,coverage_breadth,expected,aligned,is_s_phase,is_normal,is_wgd
0,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM-130057A-...,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM,130057A,SPECTRUM-OV-002,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM_130057A_L1,SPECTRUM-OV-002-OMENTUM,3338405,2889287,3158911,0.067330,0.063475,0.134170,0.120091,False,False,1
1,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM-130057A-...,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM,130057A,SPECTRUM-OV-002,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM_130057A_L1,SPECTRUM-OV-002-OMENTUM,3074108,3315895,3607552,0.074933,0.070339,0.153470,0.138260,False,False,1
2,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM-130057A-...,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM,130057A,SPECTRUM-OV-002,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM_130057A_L1,SPECTRUM-OV-002-OMENTUM,2783642,1269548,1384529,0.031480,0.030321,0.058491,0.052290,False,False,1
3,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM-130057A-...,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM,130057A,SPECTRUM-OV-002,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM_130057A_L1,SPECTRUM-OV-002-OMENTUM,3274462,3365498,3627216,0.075640,0.071320,0.154042,0.139890,False,True,0
4,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM-130057A-...,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM,130057A,SPECTRUM-OV-002,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM_130057A_L1,SPECTRUM-OV-002-OMENTUM,2748275,2880537,3153862,0.057989,0.054099,0.129631,0.114774,False,False,1


In [4]:
# Get normal non-Sphase cells only
normal_cells = filtered_cell_subtable[
    (filtered_cell_subtable["is_normal"] == True) &
    (filtered_cell_subtable["is_s_phase"] == False) &
    (filtered_cell_subtable["is_wgd"] == False)
    ]

# Get cell barcodes
normal_cells = normal_cells.assign(
    cell_barcode = normal_cells["cell_id"].astype(str).apply(lambda x: "-".join(x.split("-")[-3:]))
)

print(normal_cells.shape)
print(normal_cells["cell_id"].iloc[0])
normal_cells.head()

(12352, 17)
SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM-130057A-R13-C20


,cell_id,sample_id,library_id,patient_id,aliquot_id,condition,estimated_library_size,total_mapped_reads,total_reads,coverage_depth,coverage_breadth,expected,aligned,is_s_phase,is_normal,is_wgd,cell_barcode
3,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM-130057A-...,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM,130057A,SPECTRUM-OV-002,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM_130057A_L1,SPECTRUM-OV-002-OMENTUM,3274462,3365498,3627216,0.075640,0.071320,0.154042,0.139890,False,True,0,130057A-R13-C20
5,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM-130057A-...,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM,130057A,SPECTRUM-OV-002,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM_130057A_L1,SPECTRUM-OV-002-OMENTUM,1938713,1696309,1955522,0.037839,0.036380,0.082191,0.068574,False,True,0,130057A-R13-C25
8,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM-130057A-...,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM,130057A,SPECTRUM-OV-002,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM_130057A_L1,SPECTRUM-OV-002-OMENTUM,2273080,2460846,2853072,0.053377,0.051012,0.120544,0.099807,False,True,0,130057A-R13-C44
12,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM-130057A-...,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM,130057A,SPECTRUM-OV-002,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM_130057A_L1,SPECTRUM-OV-002-OMENTUM,1993489,1932309,2248005,0.041391,0.039795,0.093698,0.077278,False,True,0,130057A-R13-C55
14,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM-130057A-...,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM,130057A,SPECTRUM-OV-002,SPECTRUM-OV-002_S1_INFRACOLIC_OMENTUM_130057A_L1,SPECTRUM-OV-002-OMENTUM,2451520,2150841,2498277,0.048732,0.046625,0.105834,0.087222,False,True,0,130057A-R13-C58


In [5]:
# Get list of unique aliquots 
aliquots = normal_cells["aliquot_id"].value_counts()
aliquot_data_dict = get_data_paths(aliquots.index.tolist())

Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 2 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 0 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...


Search for SPECTRUM-OV-051_S1_NUCLEI_RIGHT_ADNEXA_NEW_128438A_L1 returned 0 results. Please check the analysis.


Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 2 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 0 from analyses API endpoint...
Retrieving 0 from analyses API endpoint...
Retrieving 0 from analyses API endpoint...


Search for SPECTRUM-OV-087_S1_UNSORTED_B_INFRACOLIC_OMENTUM_130032A_L1 returned 0 results. Please check the analysis.
Search for SPECTRUM-OV-051_S1_UNSORTED_INFRACOLIC_OMENTUM_130040A_L1 returned 0 results. Please check the analysis.
Search for SPECTRUM-OV-087_S1_UNSORTED_A_INFRACOLIC_OMENTUM_130032A_L1 returned 0 results. Please check the analysis.


Retrieving 2 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 2 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 0 from analyses API endpoint...


...

Retrieving 1 from analyses API endpoint...
Retrieving 2 from analyses API endpoint...
Retrieving 2 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 2 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 2 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 2 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 2 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 

Search for SPECTRUM-OV-065_S1_UNSORTED_RIGHT_OVARY_130380A_L1 returned 0 results. Please check the analysis.


Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...
Retrieving 0 from analyses API endpoint...
Retrieving 0 from analyses API endpoint...


Search for SPECTRUM-OV-065_S1_CD45N_RIGHT_OVARY_130021A_L1 returned 0 results. Please check the analysis.
Search for SPECTRUM-OV-051_S1_NUCLEI_RIGHT_ADNEXA_OLD_128438A_L1 returned 0 results. Please check the analysis.


Retrieving 1 from analyses API endpoint...
Retrieving 1 from analyses API endpoint...


In [6]:
# Make dataframe with columns: aliquot_id, bam_path, sample_id, total_cell_count, normal_cell_count
all_samples_df = pd.DataFrame(aliquot_data_dict, columns=["aliquot_id", "bam_path", "gc_metrics_path"])
all_samples_df = all_samples_df.assign(
    sample_name = all_samples_df["bam_path"].astype(str).apply(lambda x: "_".join(x.split("/")[-1].split("_")[:-3])),
    total_cell_count = all_samples_df["aliquot_id"].astype(str).apply(
        lambda x: filtered_cell_table[filtered_cell_table["aliquot_id"] == x].shape[0]
    ),
    normal_cell_count = all_samples_df["aliquot_id"].astype(str).apply(
        lambda x: normal_cells[normal_cells["aliquot_id"] == x].shape[0]
    ),
)

all_samples_df

,aliquot_id,bam_path,gc_metrics_path,sample_name,total_cell_count,normal_cell_count
0,SPECTRUM-OV-105_S1_UNSORTED_INFRACOLIC_OMENTUM...,/data1/shahs3/isabl_data_lake/analyses/90/67/2...,/data1/shahs3/isabl_data_lake/analyses/90/67/2...,SHAH_H000705_T02_04_DLP01,1349,741
1,SPECTRUM-OV-049_S1_UNSORTED_INFRACOLIC_OMENTUM...,/data1/shahs3/isabl_data_lake/analyses/90/69/2...,/data1/shahs3/isabl_data_lake/analyses/90/69/2...,SHAH_H000036_T01_03_DLP01,1385,622
2,SPECTRUM-OV-110_S1_CD45N_BOWEL_130424A_L2,/data1/shahs3/isabl_data_lake/analyses/90/85/2...,/data1/shahs3/isabl_data_lake/analyses/90/85/2...,SHAH_H000706_T03_05_DLP01,603,592
3,SPECTRUM-OV-045_S1_CD45N_RIGHT_OVARY_A98245B_L5,/data1/shahs3/isabl_data_lake/analyses/91/19/2...,/data1/shahs3/isabl_data_lake/analyses/91/19/2...,SHAH_H000031_T05_03_DLP01,944,570
4,SPECTRUM-OV-004_S1_UNSORTED_INFRACOLIC_OMENTUM...,/data1/shahs3/isabl_data_lake/analyses/90/99/2...,/data1/shahs3/isabl_data_lake/analyses/90/99/2...,SHAH_H000029_T03_01_DLP01,1335,562
...,...,...,...,...,...,...
69,SPECTRUM-OV-022_S1_CD45N_LEFT_ADNEXA_A108833A_L2,/data1/shahs3/isabl_data_lake/analyses/91/30/2...,/data1/shahs3/isabl_data_lake/analyses/91/30/2...,SHAH_H000019_T05_05_DLP01,319,8
70,SPECTRUM-OV-051_S1_CD45N_BOWEL_110675_L1,/data1/shahs3/isabl_data_lake/analyses/24/44/3...,/data1/shahs3/isabl_data_lake/analyses/24/44/3...,SHAH_H000034_T01_02_DLP01,42,8
71,SPECTRUM-OV-044_S1_UNSORTED_RIGHT_FALLOPIAN_TU...,/data1/shahs3/isabl_data_lake/analyses/90/92/2...,/data1/shahs3/isabl_data_lake/analyses/90/92/2...,SHAH_H000162_T04_01_DLP01,734,7
72,SPECTRUM-OV-065_S1_UNSORTED_RIGHT_OVARY_128655...,/data1/shahs3/isabl_data_lake/analyses/24/46/3...,/data1/shahs3/isabl_data_lake/analyses/24/46/3...,SHAH_H000041_T04_03_DLP01,176,4


In [48]:
# General functioin to group samples into groups with 700~1000 cells
def group_samples_by_cell_count(df, max_cell_count = 1000):
    """
    Group samples by cell count.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing sample information with columns 'sample_name' and 'normal_cell_count'.
    max_cell_count : int
        Maximum cell count for each group.

    Returns
    -------
    final_groups: list
        List of groups, where each group is a list of sample IDs.
    """
    sample_counts_lst = list(df[["sample_name", "normal_cell_count"]].itertuples(index=False, name=None))

    final_groups = []
    final_counts = []
    while len(sample_counts_lst) > 0:
        if sample_counts_lst[0][1] > max_cell_count:
            final_groups.append(sample_counts_lst[0][0])
            final_counts.append(sample_counts_lst[0][1])
            sample_counts_lst.remove(sample_counts_lst[0])
        else:
            current_group = [sample_counts_lst[0][0]]
            current_count = sample_counts_lst[0][1]
            sample_counts_lst.remove(sample_counts_lst[0])
            while current_count < max_cell_count and len(sample_counts_lst) > 0:
                if sample_counts_lst[-1][1] + current_count < max_cell_count:
                    current_group.append(sample_counts_lst[-1][0])
                    current_count += sample_counts_lst[-1][1]
                    sample_counts_lst.remove(sample_counts_lst[-1])
                else:
                    break
            final_groups.append(current_group)
            final_counts.append(current_count)
    print(f"Final counts: {final_counts}")
    print(f"Final groups: {final_groups}")
    return final_groups


sample_groups = group_samples_by_cell_count(all_samples_df)

Final counts: [976, 991, 936, 925, 983, 888, 869, 849, 881, 841, 865, 886, 734]
Final groups: [['SHAH_H000705_T02_04_DLP01', 'SHAH_H000034_T07_03_DLP01', 'SHAH_H000041_T04_03_DLP01', 'SHAH_H000162_T04_01_DLP01', 'SHAH_H000034_T01_02_DLP01', 'SHAH_H000019_T05_05_DLP01', 'SHAH_H000340_T01_04_DLP01', 'SHAH_H000019_T07_04_DLP01', 'SHAH_H000023_T02_03_DLP01', 'SHAH_H000013_T03_04_DLP01', 'SHAH_H000006_T10_02_DLP01', 'SHAH_H000005_T04_04_DLP01', 'SHAH_H000339_T03_04_DLP01', 'SHAH_H000019_T05_04_DLP01', 'SHAH_H000222_T01_06_DLP01', 'SHAH_H000022_T14_02_DLP01', 'SHAH_H000338_T04_03_DLP01'], ['SHAH_H000036_T01_03_DLP01', 'SHAH_H000013_T06_03_DLP01', 'SHAH_H000031_T01_04_DLP01', 'SHAH_H000013_T15_02_DLP01', 'SHAH_H000024_T02_03_DLP01', 'SHAH_H000041_T02_04_DLP01', 'SHAH_H000342_T04_04_DLP01', 'SHAH_H000035_T03_03_DLP01', 'SHAH_H000222_T01_05_DLP01', 'SHAH_H000005_T03_06_DLP01'], ['SHAH_H000706_T03_05_DLP01', 'SHAH_H000006_T15_04_DLP01', 'SHAH_H000039_T11_03_DLP01', 'SHAH_H000340_T03_03_DLP01', '

In [49]:
# Add new column to dataframe with group number
all_samples_df = all_samples_df.assign(
    sample_group = all_samples_df["sample_name"].astype(str).apply(
        lambda x: [group_num for (group_num, group_lst) in enumerate(sample_groups) if x in group_lst][0]
    )
)

In [50]:
all_samples_df

,aliquot_id,bam_path,gc_metrics_path,sample_name,total_cell_count,normal_cell_count,sample_group
0,SPECTRUM-OV-105_S1_UNSORTED_INFRACOLIC_OMENTUM...,/data1/shahs3/isabl_data_lake/analyses/90/67/2...,/data1/shahs3/isabl_data_lake/analyses/90/67/2...,SHAH_H000705_T02_04_DLP01,1349,741,0
1,SPECTRUM-OV-049_S1_UNSORTED_INFRACOLIC_OMENTUM...,/data1/shahs3/isabl_data_lake/analyses/90/69/2...,/data1/shahs3/isabl_data_lake/analyses/90/69/2...,SHAH_H000036_T01_03_DLP01,1385,622,1
2,SPECTRUM-OV-110_S1_CD45N_BOWEL_130424A_L2,/data1/shahs3/isabl_data_lake/analyses/90/85/2...,/data1/shahs3/isabl_data_lake/analyses/90/85/2...,SHAH_H000706_T03_05_DLP01,603,592,2
3,SPECTRUM-OV-045_S1_CD45N_RIGHT_OVARY_A98245B_L5,/data1/shahs3/isabl_data_lake/analyses/91/19/2...,/data1/shahs3/isabl_data_lake/analyses/91/19/2...,SHAH_H000031_T05_03_DLP01,944,570,3
4,SPECTRUM-OV-004_S1_UNSORTED_INFRACOLIC_OMENTUM...,/data1/shahs3/isabl_data_lake/analyses/90/99/2...,/data1/shahs3/isabl_data_lake/analyses/90/99/2...,SHAH_H000029_T03_01_DLP01,1335,562,4
...,...,...,...,...,...,...,...
69,SPECTRUM-OV-022_S1_CD45N_LEFT_ADNEXA_A108833A_L2,/data1/shahs3/isabl_data_lake/analyses/91/30/2...,/data1/shahs3/isabl_data_lake/analyses/91/30/2...,SHAH_H000019_T05_05_DLP01,319,8,0
70,SPECTRUM-OV-051_S1_CD45N_BOWEL_110675_L1,/data1/shahs3/isabl_data_lake/analyses/24/44/3...,/data1/shahs3/isabl_data_lake/analyses/24/44/3...,SHAH_H000034_T01_02_DLP01,42,8,0
71,SPECTRUM-OV-044_S1_UNSORTED_RIGHT_FALLOPIAN_TU...,/data1/shahs3/isabl_data_lake/analyses/90/92/2...,/data1/shahs3/isabl_data_lake/analyses/90/92/2...,SHAH_H000162_T04_01_DLP01,734,7,0
72,SPECTRUM-OV-065_S1_UNSORTED_RIGHT_OVARY_128655...,/data1/shahs3/isabl_data_lake/analyses/24/46/3...,/data1/shahs3/isabl_data_lake/analyses/24/46/3...,SHAH_H000041_T04_03_DLP01,176,4,0


In [68]:
# Save list of all normal cells
filtered_normal_cells = normal_cells[normal_cells["aliquot_id"].isin(all_samples_df["aliquot_id"].tolist())]
filtered_normal_cells = filtered_normal_cells[["cell_id", "sample_id", "library_id", "patient_id", "aliquot_id", "condition",
                                               "estimated_library_size", "total_mapped_reads", "total_reads", 
                                               "coverage_depth", "coverage_breadth", "expected", "aligned",
                                               "cell_barcode"]]
print(filtered_normal_cells.shape)

filtered_normal_cells = pd.merge(
    filtered_normal_cells,
    all_samples_df[["aliquot_id", "sample_name", "sample_group"]],
    on = "aliquot_id",
    how = "left"
)

filtered_normal_cells = filtered_normal_cells.sort_values(by=["sample_group", "sample_name", "cell_barcode"]).reset_index(drop=True)

filtered_normal_cells


(11624, 14)


,cell_id,sample_id,library_id,patient_id,aliquot_id,condition,estimated_library_size,total_mapped_reads,total_reads,coverage_depth,coverage_breadth,expected,aligned,cell_barcode,sample_name,sample_group
0,SPECTRUM-OV-003_S1_PELVIC_PERITONEUM-A96167B-R...,SPECTRUM-OV-003_S1_PELVIC_PERITONEUM,A96167B,SPECTRUM-OV-003,SPECTRUM-OV-003_S1_UNSORTED_PELVIC_PERITONEUM_...,PP,5453476,3392789,3508534,0.059263,0.056497,0.084449,0.080784,A96167B-R54-C55,SHAH_H000005_T04_04_DLP01,0
1,SPECTRUM-OV-003_S1_PELVIC_PERITONEUM-A96167B-R...,SPECTRUM-OV-003_S1_PELVIC_PERITONEUM,A96167B,SPECTRUM-OV-003,SPECTRUM-OV-003_S1_UNSORTED_PELVIC_PERITONEUM_...,PP,5411886,3538071,3790349,0.061562,0.058540,0.091234,0.084117,A96167B-R54-C56,SHAH_H000005_T04_04_DLP01,0
2,SPECTRUM-OV-003_S1_PELVIC_PERITONEUM-A96167B-R...,SPECTRUM-OV-003_S1_PELVIC_PERITONEUM,A96167B,SPECTRUM-OV-003,SPECTRUM-OV-003_S1_UNSORTED_PELVIC_PERITONEUM_...,PP,4242144,2704402,2759114,0.046620,0.044603,0.066333,0.064347,A96167B-R54-C59,SHAH_H000005_T04_04_DLP01,0
3,SPECTRUM-OV-003_S1_PELVIC_PERITONEUM-A96167B-R...,SPECTRUM-OV-003_S1_PELVIC_PERITONEUM,A96167B,SPECTRUM-OV-003,SPECTRUM-OV-003_S1_UNSORTED_PELVIC_PERITONEUM_...,PP,3647525,2523703,2583990,0.042741,0.040969,0.062098,0.060012,A96167B-R54-C60,SHAH_H000005_T04_04_DLP01,0
4,SPECTRUM-OV-003_S1_PELVIC_PERITONEUM-A96167B-R...,SPECTRUM-OV-003_S1_PELVIC_PERITONEUM,A96167B,SPECTRUM-OV-003,SPECTRUM-OV-003_S1_UNSORTED_PELVIC_PERITONEUM_...,PP,3714769,2617686,2658424,0.044475,0.042611,0.063898,0.062272,A96167B-R54-C66,SHAH_H000005_T04_04_DLP01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11619,SPECTRUM-OV-133_S1_LEFT_FALLOPIAN_TUBE-128708A...,SPECTRUM-OV-133_S1_LEFT_FALLOPIAN_TUBE,128708A,SPECTRUM-OV-133,SPECTRUM-OV-133_S1_UNSORTED_LEFT_FALLOPIAN_TUB...,133LFT_DLP_UNSORTED,3268668,5040795,5083438,0.099401,0.093133,0.213987,0.209239,128708A-R61-C50,SHAH_H002023_T03_01_DLP01,12
11620,SPECTRUM-OV-133_S1_LEFT_FALLOPIAN_TUBE-128708A...,SPECTRUM-OV-133_S1_LEFT_FALLOPIAN_TUBE,128708A,SPECTRUM-OV-133,SPECTRUM-OV-133_S1_UNSORTED_LEFT_FALLOPIAN_TUB...,133LFT_DLP_UNSORTED,2841409,4281928,4305410,0.087972,0.082872,0.183577,0.180273,128708A-R61-C52,SHAH_H002023_T03_01_DLP01,12
11621,SPECTRUM-OV-133_S1_LEFT_FALLOPIAN_TUBE-128708A...,SPECTRUM-OV-133_S1_LEFT_FALLOPIAN_TUBE,128708A,SPECTRUM-OV-133,SPECTRUM-OV-133_S1_UNSORTED_LEFT_FALLOPIAN_TUB...,133LFT_DLP_UNSORTED,2420298,2719094,2741092,0.064917,0.061840,0.121958,0.119221,128708A-R61-C55,SHAH_H002023_T03_01_DLP01,12
11622,SPECTRUM-OV-133_S1_LEFT_FALLOPIAN_TUBE-128708A...,SPECTRUM-OV-133_S1_LEFT_FALLOPIAN_TUBE,128708A,SPECTRUM-OV-133,SPECTRUM-OV-133_S1_UNSORTED_LEFT_FALLOPIAN_TUB...,133LFT_DLP_UNSORTED,3325636,4788084,4832454,0.096574,0.090661,0.204270,0.199188,128708A-R61-C57,SHAH_H002023_T03_01_DLP01,12


In [ ]:
# Save the dataframe of sample groups to a CSV file
all_samples_df_path = f"{DATADIR}/all_normal_samples.csv"
# all_samples_df.to_csv(all_samples_df_path, index=False)

In [69]:
# Save dataframe of all filtered normal cells to a CSV file
filtered_normal_cells_path = f"{DATADIR}/filtered_normal_cells.csv"
filtered_normal_cells.to_csv(filtered_normal_cells_path, index=False)

In [70]:
# For each sample, save the filtered normal cell barcodes to a text file
CELL_LIST_DIR = f"{DATADIR}/normal_cell_barcodes"

for i in range(len(all_samples_df)):
    aliquot_id = all_samples_df["aliquot_id"].iloc[i]
    sample_name = all_samples_df["sample_name"].iloc[i]

    cell_barcodes = filtered_normal_cells[filtered_normal_cells["aliquot_id"] == aliquot_id]["cell_barcode"].tolist()
    cell_barcode_path = f"{CELL_LIST_DIR}/{sample_name}_cell_barcodes.txt"

    with open(cell_barcode_path, "w") as f:
        f.write("\n".join(cell_barcodes))

In [67]:
# Get GC metrics for each sample and group by sample group
def aggregate_gc_metrics(all_cells_df, all_samples_df, group_num):
    samples_df = all_samples_df[all_samples_df["sample_group"] == group_num]
    cells_df = all_cells_df[all_cells_df["sample_group"] == group_num]

    combined_gc_metrics_df = pd.DataFrame()
    for index, row in samples_df.iterrows():
        sample_name = row["sample_name"]
        gc_metrics_path = row["gc_metrics_path"]

        # Get GC metrics
        gc_metrics_df = pd.read_csv(gc_metrics_path)
        gc_metrics_df = gc_metrics_df[gc_metrics_df["cell_id"].isin(cells_df["cell_barcode"].tolist())]
        gc_metrics_df = gc_metrics_df.assign(
            sample_name = sample_name,
        )
        gc_metrics_df.rename(columns = {"cell_id" : "cell_barcode"}, inplace = True)
        combined_gc_metrics_df = pd.concat([combined_gc_metrics_df, gc_metrics_df], ignore_index=True)

    combined_gc_metrics_df = combined_gc_metrics_df.sort_values(by=["sample_name", "cell_barcode"]).reset_index(drop=True)

    return combined_gc_metrics_df


# Get GC metrics for each sample group and save
for group_num in all_samples_df["sample_group"].unique():
    gc_metrics_path = f"{DATADIR}/normal_cell_gc_metrics/group_{group_num}_gc_metrics.csv"
    group_gc_metrics_df = aggregate_gc_metrics(filtered_normal_cells, all_samples_df, group_num)
    group_gc_metrics_df.to_csv(gc_metrics_path, index=False)

In [8]:
# Get chromosome lengths

bam_file = pysam.AlignmentFile(all_samples_df["bam_path"].iloc[0], "rb")
chr_lengths = dict(zip(bam_file.references, bam_file.lengths))

chr_lengths_df = pd.DataFrame(list(chr_lengths.items()), columns = ["chr", "length"])
chr_lengths_df.to_csv(f"{DATADIR}/chr_lengths.csv", index = False)

[W::hts_idx_load3] The index file is older than the data file: /data1/shahs3/isabl_data_lake/analyses/90/67/29067/results/SHAH_H000705_T02_04_DLP01_all_cells_bulk.bam.bai
